# Keras LSTM Text Generation
Text generation is a important nlp problem which can enable computers to write.

<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/github/TannerGilbert/Tutorials/blob/master/Keras-Tutorials/4.%20LSTM%20Text%20Generation/Keras%20LSTM%20Text%20Generation.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td><td>
  <a target="_blank"  href="https://github.com/TannerGilbert/Tutorials/blob/master/Keras-Tutorials/4.%20LSTM%20Text%20Generation/Keras%20LSTM%20Text%20Generation.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
</td></table>

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [4]:
text = open('sherlock_homes.txt', 'r').read().lower()
print('text length', len(text))

text length 561852


In [5]:
print(text[:300])

ï»¿adventure i. a scandal in bohemia

i.

to sherlock holmes she is always the woman. i have seldom heard
him mention her under any other name. in his eyes she eclipses
and predominates the whole of her sex. it was not that he felt
any emotion akin to love for irene adler. all emotions, and that
one


## Map chars to integers

In [10]:
chars = sorted(list(set(text)))
print('total chars: ', len(chars))
print(chars)

total chars:  59
['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '¢', '¨', '©', '»', '¿', 'ã', 'ï']


In [13]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

## Split up into subsequences

In [8]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 187271


In [9]:
print(sentences[:3])
print(next_chars[:3])

['ï»¿adventure i. a scandal in bohemia\n\ni.', 'adventure i. a scandal in bohemia\n\ni.\n\nt', 'enture i. a scandal in bohemia\n\ni.\n\nto s']
['\n', 'o', 'h']


In [14]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [15]:
print(x[:3])
print(y[:3])

[[[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ...  True False False]
  ...
  [ True False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [ True False False ... False False False]
  [ True False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]]]
[[ True False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

## Building Model

In this notebook a small recurrent neural networks is used for both simplicity and because of the training time but if you want to train a more sophisticated model you can increase the size of the network. You can also use a model pretrained on some other text like wikipedia text to both speed up the training process and get better results.

In [16]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [17]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print("Compiled!")

Compiled!


## Helper Functions
These helper functions are taken from the [official Keras text generation notebook](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py).

In [18]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [19]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

## Defining callbacks

In [20]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [21]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

In [22]:
callbacks = [print_callback, checkpoint, reduce_lr]

## Training the model

In [23]:
model.fit(x, y, batch_size=128, epochs=1, callbacks=callbacks)

Epoch 1/1
187271/187271 [==============================] - 318s 2ms/step - loss: 1.9532

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "of a
morning."

"that will do, mr. wilso"
of a
morning."

"that will do, mr. wilson, and of a rears of the man in the that i have the man in a stanted the man in the man the round of the other in the marted in the little in the tount of the man a man a stantle a deen the fittle and streathed it in the man of the come the complay the fittle and hand in the intere and the sime that i had a stan of a little in the man a mind the man his handed the man in the man in the man a man t
----- diversity: 0.5
----- Generating with seed: "of a
morning."

"that will do, mr. wilso"
of a
morning."

"that will do, mr. wilso in a cain it i have neirs fing it mind the down my presed the hone of the mind a string through it was a a mest of the man herred the dreed a moment i shand and a part in the doors of the mades and the streat

## Testing the model
Now that we have a trained network we can test it using a method simular to the ``on_epoch_end`` method above.

In [28]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [29]:
print(generate_text(500, 0.2))

remarked, looking up in surprise.

"i must the lott of the man the mind the man in the man in the simple of the rear and a stander of the stark that i had not the mind the door and in the man in the man and and the sill of a reard of the that i come the man a shand of the door holmes it is a come the man a do the that i have the simple that i have a pose that i made the preared and a standed the sime the door and the simple and a stant of the man in the lotted of the little a man that i am the wind of the man of the ficed the little t
